In [1]:
from brainbox.task import trials
from one.api import ONE
import pandas as pd
import numpy as np
import GEMA

one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)
eid = '58b1e920-cfc8-467e-b28b-7654a55d0977'



In [2]:

def get_raster_and_psth(eid):
    #Load trials into df
    dat = one.load_dataset(eid, 'alf/_ibl_trials.table.pqt')
    df=pd.DataFrame(dat)
    #Choose one type of event
    trials_=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 1.0)].index
    #df=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 1.0)]
    events=df["stimOn_times"]
    #Get spike_times
    spike_times=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.times.npy')
    #Get clusters
    spike_clusters=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.clusters.npy')
    print(len(spike_clusters))
    print(spike_times.shape)
    #Get binned rasters
    raster_dict={}
    time_vec={}
    for n in list(np.unique(spike_clusters)):
        neuron=np.where(spike_clusters==n)
        if len(list(neuron[0]))!=0:
            #print(len(list(neuron[0])))
            raster=trials.get_event_aligned_raster(spike_times[neuron], events, tbin=0.1, values=None, epoch=[-0.5, 2], bin=True)
            #print(raster)
            if np.isnan(raster[0]).any()==False:
                raster_dict[n]=raster[0]
        time_vec=raster[1]
    psth_dict={}
    for k in raster_dict.keys():
        psth=trials.get_psth(raster_dict[k],trial_ids=trials_)
        psth_dict[k]=psth
    return raster_dict,psth_dict,time_vec

raster_dict, psth_dict,time_vec=get_raster_and_psth(eid)

5469038
(5469038,)


In [3]:
psth_mat=[]
for k in psth_dict.keys():
    psth_mat.append(psth_dict[k][0])
psth_mat=np.array(psth_mat)

In [4]:
print(psth_mat.shape)

(256, 26)


In [11]:
np.save('psth_arr.npy',psth_mat)

In [5]:
import GEMA

import time

start=time.time()
print(start)

map_size = 16
period = 10000 
learning_rate = 0.1
normalize_data = 'none'

main_map = GEMA.Map(psth_mat,
                    map_size,
                    period, 
                    initial_lr=learning_rate, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    normalization = normalize_data,
                    presentation= 'random',
                    weights = 'random')

end=time.time()
print(end-start)

plt_arr=[]
for j in range(0,16):
    plt_lst=[]
    for i in range(0,16):
        plt_lst.append(main_map.weights[i,j,:])
    plt_arr.append(plt_lst)
plt_arr=np.array(plt_arr)
print(plt_arr.shape)
np.save('psth_prototypes_arr.npy',plt_arr)

1678589967.6664243
TRAINING...


  0%|          | 0/10000 [00:00<?, ?it/s]

FINISHED.
1.9741108417510986
(16, 16, 26)


In [6]:
plt_arr=[]
for j in range(0,16):
    plt_lst=[]
    for i in range(0,16):
        plt_lst.append(main_map.weights[i,j,:])
    plt_arr.append(plt_lst)
plt_arr=np.array(plt_arr)
print(plt_arr.shape)
np.save('psth_prototypes_arr.npy',plt_arr)

(16, 16, 26)


In [ ]:
print(alpha_im_z.sh)

In [7]:
main_map.calculate_bmu(psth_mat[0,:])[1]

(15, 0)

In [10]:
import pickle
bmu_dct={}
for i in range(0,16):
    for j in range(0,16):
        bmu_dct[(i,j)]=[]
for k in range(0,psth_mat.shape[0]):
    grid_index=main_map.calculate_bmu(psth_mat[k,:])[1]
    bmu_dct[grid_index].append(k)
with open('bmu_dct.pickle', 'wb') as handle:
    pickle.dump(bmu_dct, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(bmu_dct)

{(0, 0): [154], (0, 1): [], (0, 2): [62, 67], (0, 3): [], (0, 4): [72], (0, 5): [144], (0, 6): [63, 114], (0, 7): [207, 232], (0, 8): [58, 135, 149, 150], (0, 9): [], (0, 10): [136, 171], (0, 11): [148, 206], (0, 12): [37, 164], (0, 13): [172, 204, 248], (0, 14): [241], (0, 15): [16, 22, 27, 66, 78, 87, 88, 90, 95, 98, 146, 155, 157, 210, 211, 212, 221, 224, 225, 226, 227, 230, 233, 239, 247, 249, 250, 252], (1, 0): [140], (1, 1): [], (1, 2): [183], (1, 3): [20], (1, 4): [89], (1, 5): [240], (1, 6): [], (1, 7): [], (1, 8): [], (1, 9): [], (1, 10): [32], (1, 11): [19, 91], (1, 12): [57, 65], (1, 13): [173, 185, 234], (1, 14): [159, 168, 208, 237], (1, 15): [40, 223], (2, 0): [], (2, 1): [], (2, 2): [108], (2, 3): [], (2, 4): [33], (2, 5): [107, 176], (2, 6): [], (2, 7): [], (2, 8): [122], (2, 9): [55, 106, 182], (2, 10): [], (2, 11): [92, 197], (2, 12): [94, 243], (2, 13): [], (2, 14): [36, 216, 246], (2, 15): [35, 113, 193, 245, 254], (3, 0): [138, 139], (3, 1): [], (3, 2): [], (3, 3):

In [12]:
len(list(np.arange(-0.5,2.2,0.1)))
    

27

In [13]:
#Color SOM
from sklearn import preprocessing
datos = np.random.randint(0, 255, (100, 3))
datos = preprocessing.normalize(datos)
main_map_colors = GEMA.Map(datos, 
                    map_size, 
                    period, 
                    initial_lr=learning_rate, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    normalization = normalize_data,
                    presentation= 'random',
                    weights = 'random')
from matplotlib.pyplot import imshow
image_matrix = (main_map_colors.weights-main_map_colors.weights.min())/(main_map_colors.weights.max()-main_map_colors.weights.min())
print(image_matrix.shape)

TRAINING...


  0%|          | 0/10000 [00:00<?, ?it/s]

FINISHED.
(16, 16, 3)


In [36]:
alpha_im=np.zeros((16,16,4))
for i in range(0,16):
    for j in range(0,16):
        alpha_im[i,j,:3]=image_matrix[i,j,:]
        if len(bmu_dct[(i,j)])==0:
            alpha_im[i,j,3]=0.0
        else:
            alpha_im[i,j,3]=1

In [46]:
alpha_im=np.zeros((16,16,4))
for i in range(0,16):
    for j in range(0,16):
        alpha_im[i,j,:3]=image_matrix[i,j,:]
        if len(bmu_dct[(i,j)])==0:
            alpha_im[i,j,3]=0.5
        else:
            alpha_im[i,j,3]=1

In [21]:
bmu_dct[(15,10)]

[60]

In [22]:
bmu_dct[(12,11)]

[111, 141]

In [24]:
alpha_im[12,11,:]

array([0.52019749, 0.21400146, 0.79646675, 1.        ])

In [34]:
bmu_dct[(10,1)]

[]

In [45]:
alpha_im[15,2]

array([0.97244631, 0.03501001, 0.19696924, 0.2       ])

In [47]:
np.save('alpha_im.npy',alpha_im)

In [42]:
print(alpha_im[15,2])

[0.97244631 0.03501001 0.19696924 0.        ]


In [31]:
a=np.load('alpha_im.npy')

In [32]:
a

array([[[0.48386384, 0.86679078, 0.        , 1.        ],
        [0.45210576, 0.87758701, 0.00575845, 0.1       ],
        [0.35907639, 0.90614644, 0.02623991, 1.        ],
        ...,
        [0.03121727, 0.7910949 , 0.58245526, 1.        ],
        [0.01536084, 0.75726155, 0.62746121, 1.        ],
        [0.00723302, 0.7346602 , 0.65520681, 1.        ]],

       [[0.50901732, 0.84937019, 0.01199665, 1.        ],
        [0.48981212, 0.85705032, 0.02135281, 0.1       ],
        [0.41488873, 0.88137222, 0.03406801, 1.        ],
        ...,
        [0.04425674, 0.76913842, 0.60396324, 1.        ],
        [0.03337532, 0.72296526, 0.66164043, 1.        ],
        [0.0231903 , 0.69832105, 0.69050018, 1.        ]],

       [[0.53727245, 0.82776091, 0.02310379, 0.1       ],
        [0.5326942 , 0.82810352, 0.03446852, 0.1       ],
        [0.49532666, 0.83815376, 0.05980007, 1.        ],
        ...,
        [0.09333652, 0.70587697, 0.6642684 , 0.1       ],
        [0.07309218, 0.682089